In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

!pip install openpyxl

/kaggle/input/supersession/SuperSession.xlsx
     |████████████████████████████████| 242 kB 5.0 MB/s            


In [2]:
ss = pd.read_excel("../input/supersession/SuperSession.xlsx", engine="openpyxl")

In [3]:
# Sessão para verificar se existem as cadeias globais na lista (foi feito um Workaround para selecionar TODAS as cadeias do arquivo, logo todos serão selecionados)

ss_profile_global = pd.DataFrame(ss["GROUP_NUMBER"].unique(), columns = ['GROUP_NUMBER'])
#ss_profile_global = pd.DataFrame(["C28644_010", "C29362_010"], columns = ['GROUP_NUMBER'])

ss = ss[ss.GROUP_NUMBER.isin(ss_profile_global.GROUP_NUMBER)]

In [4]:
# Sessão para retirar a duplicidade onde um Predecessor possui mais de um Sucessor
ss_1PREC_NSUCC = pd.pivot_table(ss, values='SUCC_PRODUCT', index='PREC_PRODUCT',
               aggfunc='count')

ss_1PREC_NSUCC = ss_1PREC_NSUCC.loc[ss_1PREC_NSUCC['SUCC_PRODUCT'] > 1]

ss = ss[-ss.PREC_PRODUCT.isin(ss_1PREC_NSUCC.index)]

In [5]:
# Sessão para retirar a duplicidade onde um Sucessor possui mais de um Predecessor quando a cadeia é do tipo Duas Vias

ss_YY = ss.loc[ss["TIPO_INTERC"] == "I"]

ss_1SUCC_NPREC_YY = pd.pivot_table(ss_YY, values='PREC_PRODUCT', index='SUCC_PRODUCT',
               aggfunc='count')

ss_1SUCC_NPREC_YY = ss_1SUCC_NPREC_YY.loc[ss_1SUCC_NPREC_YY['PREC_PRODUCT'] > 1]

ss = ss[-ss.SUCC_PRODUCT.isin(ss_1SUCC_NPREC_YY.index)]

In [6]:
ss.to_excel("SuperSession_Clean.xlsx")

In [7]:
ss.head()

,GROUP_NUMBER,PREC_PRODUCT,SUCC_PRODUCT,TIPO_INTERC
0,BF2LCHD,BF2D4LCHD,BF2DK2LCHD,I
1,C00002_010,16100ZF6W20,16100ZF6W21,F
2,C00003_010,90012881000,90012882000,I
3,C00004_010,44301098000,44301107000,F
4,C00004_010,44301107000,44301098010,F


In [8]:
all_material_Predecessor = pd.DataFrame(ss["PREC_PRODUCT"].unique(), columns = ['PREC_PRODUCT'])
all_material_Successor = pd.DataFrame(ss["SUCC_PRODUCT"].unique(), columns = ['SUCC_PRODUCT'])

all_material_Predecessor.rename(columns={"PREC_PRODUCT": "Materiais"}, inplace=True)
all_material_Successor.rename(columns={"SUCC_PRODUCT": "Materiais"}, inplace=True)

all_material = pd.DataFrame()
all_material = all_material.append(all_material_Predecessor)
all_material = all_material.append(all_material_Successor)

all_material = pd.DataFrame(all_material["Materiais"].unique(), columns = ['Materiais'])

In [9]:
def Material_Predecessor(Material):
    
    if int(all_material_Successor.loc[all_material_Successor["Materiais"] == Material ].count()) >= 1: #Se Material é sucessor de uma linha
        if ss.loc[(ss["SUCC_PRODUCT"] == Material) & (ss["TIPO_INTERC"] == "I")].PREC_PRODUCT.count() == 1:
            var_return = ss.loc[(ss["SUCC_PRODUCT"] == Material) & (ss["TIPO_INTERC"] == "I")].iat[0,1] 
        else:
            var_return = Material
    elif int(all_material_Predecessor.loc[all_material_Predecessor["Materiais"] == Material].count()) >= 1: #Se Material é sucessor de uma linha
        var_return = Material 
    else:
        var_return = "0"  
    return var_return

In [10]:
def Material_Successor(Material):
    if int(all_material_Predecessor.loc[all_material_Predecessor["Materiais"] == Material].count()) >= 1:#Se Material é sucessor de uma linha
        var_return = ss.loc[(ss["PREC_PRODUCT"] == Material)].iat[0,2]

    elif int(all_material_Successor.loc[all_material_Successor["Materiais"] == Material ].count()) >= 1:
        var_return = Material

    else:
        var_return = "0"
        
    return var_return

In [11]:
def Material_Priority1(Material):
    
    Material_Anterior = Material_Predecessor(Material)
    
    if Material_Anterior == "0":
        var_return = Material_Anterior
        #print("var_return, ", var_return)
    else:
        Prioridade_1 = Material_Predecessor(Material_Anterior)
        while not (Prioridade_1 == Material_Anterior):
            #print("Rodando o While")
            Material_Anterior = Prioridade_1
            #print("Material Anterior: ", Material_Anterior)
            Prioridade_1 = Material_Predecessor(Material_Anterior)
            #print("Prioridade 1: ", Prioridade_1)
        var_return = Prioridade_1
    return var_return

In [12]:
def Material_Priorities(Material):
    Priority = pd.DataFrame()
    Priority_1 = Material_Priority1(Material)
    if Priority_1 == "0":
        Priority
    else:   
        i = 1
        j = 0
        #Priority.append(Priority_1)
        Priority = Priority.append({'Material_Predecessor': Material,
                                   'Priority': i,
                                   'Alt_Material': Priority_1}, ignore_index=True)
        Next_Material = Material_Successor(Priority_1)
        while not Next_Material == Priority.Alt_Material[j]:
            i += 1
            j += 1
            #Priority.append(Next_Material)
            Priority = Priority.append({'Material_Predecessor': Material,
                                   'Priority': i,
                                   'Alt_Material': Next_Material}, ignore_index=True)
            Next_Material = Material_Successor(Next_Material)    
    return Priority

In [13]:
Material_Priorities("15400PR3014")

,Material_Predecessor,Priority,Alt_Material
0,15400PR3014,1.0,15400P0H004
1,15400PR3014,2.0,15400P0H305
2,15400PR3014,3.0,15400PH1F02
3,15400PR3014,4.0,15400PH1F03
4,15400PR3014,5.0,15400PLC003
5,15400PR3014,6.0,15400PLC004
6,15400PR3014,7.0,15400PMET01
7,15400PR3014,8.0,15400PR3003
8,15400PR3014,9.0,15400PR3004
9,15400PR3014,10.0,15400PR3014
